In [8]:
pip install python3-discogs-client

You should consider upgrading via the 'c:\users\river\documents\transparence\discogs_project\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import discogs_client

In [3]:
# Replace the following user string and user_token first
# https://python3-discogs-client.readthedocs.io/en/latest/authentication.html
# ^ Check it and register an account on Discogs.com
d = discogs_client.Client('Your arbitrary user string here', user_token='your user token here')

In [4]:
class MyTrack:
    """
    One Instance represents one row in the final excel table.
    """
    def __init__(self, title, album, posi):
        self.title = title
        self.track_artist = set()
        self.album = album
        self.labels = set()
        self.release_year = None
        self.instruments = set()
        self.genre = set()
        self.format = set()
        self.countries_of_release = set()
        self.catalog_numbers = set()
        self.requires_manual_check = False
        self.id_str = title + " in " + album
        self.url = None
        self.position = posi
        self.track_specific_info = set()
        self.release_id = None

    def __str__ (self):
        return self.title + ' ' + self.album + ' ' + str(self.release_year)

    def __repr__ (self):
        return self.title + ' ' + self.album + ' ' + str(self.release_year)

In [5]:
class MyAlbum:
    def __init__(self, title):
        self.album_title = title
        self.track_specific_info = set()
        self.tracklist = []
        self.version_ids = set()
        self.has_track_specific_info_in_release_credits = False

    def __str__ (self):
        return self.album_title

    def __repr__ (self):
        return self.album_title

In [6]:
# 108475 is the artist id for Greg Phillinganes.
greg = d.artist(108475)
# All the releases/masters related to Greg Phillinganes.
greg_rel = greg.releases
# The release list can be paginated
greg_rel_page0 = greg_rel.page(0)
greg_rel_page5 = greg_rel.page(5)


In [21]:
# Final results will be stored in this dictionary
# Key: Each track's title
# Value: a MyTrack obj
# track_dic = {}
# Name to be searched
artist_name = greg.name
# The artist instance
artist_obj = greg

# List of albums
album_list = []

added_track_set = set()


def create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album):
    """
    Create a new MyTrack instance based on the track_entry in the cur_release.
    :param cur_album:
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: title of the new track
    """
    if main_release_name == "":
        main_release_name = cur_release.title.strip()
    track_tmp = MyTrack(track_entry.title, main_release_name, track_entry.position)
    added_track_set.add(track_entry.title + " in " + main_release_name)
    track_tmp.url = cur_release.url
    track_tmp.release_id = cur_release.id
    for artist_entry in cur_release.artists:
        track_tmp.track_artist.add(artist_entry.name)
    for label_entry in cur_release.labels:
        track_tmp.labels.add(label_entry.name)
        track_tmp.catalog_numbers.add(label_entry.data['catno'])

    track_tmp.release_year = cur_release.year
    if is_release_main_artist:
        track_tmp.instruments.add("Main")
    if is_in_credits or artist_obj in track_entry.credits:
        for extra_artist_entry in cur_release.data['extraartists']:
            if extra_artist_entry['name'] == artist_name:
                track_tmp.instruments.add(extra_artist_entry['role'])
                if extra_artist_entry['tracks'] != "":
                    track_tmp.requires_manual_check = True
                    track_tmp.track_specific_info.add(extra_artist_entry['role'] + " -> tracks: " + extra_artist_entry['tracks'])
                    cur_album.track_specific_info.add("in releaseID: " + str(cur_release.id) + ": " + extra_artist_entry['role'] + " -> tracks: " + extra_artist_entry['tracks'])
                    cur_album.has_track_specific_info_in_release_credits = True
    track_tmp.genre.update(cur_release.genres)
    for format_entry in cur_release.formats:
        track_tmp.format.add(format_entry['name'])
    track_tmp.countries_of_release.add(cur_release.country)
    track_tmp.duration = track_entry.duration
    cur_album.tracklist.append(track_tmp)
    return track_tmp


def check_single_track_credit(cur_release, track_entry, artist_t, is_release_main_artist, is_in_credits, main_release_name, cur_album):
    """
    If the artist_obj is not the main artist of the release, then check artist_obj's contribution in a single track.
    :param cur_album:
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param artist_t: artist instance to be found
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: title of the current track, whether updated
    """
    if artist_t in track_entry.credits:
        cur_track_obj = create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album)
        for ex_artist_entry in track_entry.data['extraartists']:
            if ex_artist_entry['name'] == artist_name:
                new_role = ex_artist_entry['role']
                cur_track_obj.instruments.add(new_role)
        return cur_track_obj, True
    return None, False


def get_info_by_single_track(cur_release, is_release_main_artist, is_in_credits, main_release_name, cur_album):
    """
    If the artist_obj is not the main artist of the release, then check artist_obj's contribution track by track.
    :param cur_album:
    :param cur_release: current version of the master
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: null
    """
    for track_e in cur_release.tracklist:
        check_single_track_credit(cur_release, track_e, artist_obj, is_release_main_artist, is_in_credits, main_release_name, cur_album)



def get_info_from_release(cur_release, main_release_name):
    """
    Complete data retrieving in one version/release.
    :param cur_release: current version of the master
    :param main_release_name: the title of the main_release of the master
    :return: list of created MyTrack instances' titles, whether current artist is the main artist || current artist appear in "Credits"
    """
    is_release_main_artist = False
    is_in_credits = False
    cur_album = MyAlbum(cur_release.title.strip())
    cur_album.version_ids.add(cur_release.id)
    album_list.append(cur_album)
    for main_artist_e in cur_release.artists:
        if main_artist_e.id == artist_id:
            is_release_main_artist = True
            break
    for credit_artist_e in cur_release.credits:
        if credit_artist_e.id == artist_id:
            is_in_credits = True
            break
    if is_release_main_artist or is_in_credits:
        for track_entry in cur_release.tracklist:
            create_new_mytrack(main_release_tmp, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album)
        return cur_album, True
    else:
        get_info_by_single_track(cur_release, is_release_main_artist, is_in_credits, main_release_name, cur_album)
        return cur_album, False

def update_from_other_release_version(cur_release, updated_all_in_main_release, main_release_name, cur_album):
    """
    Update MyTrack in versions which are not main_release/version.
    :param cur_album:
    :param cur_release: current version of the master
    :param updated_all_in_main_release: in main_release of the master, whether current artist is the main artist || current artist appear in "Credits"
    :param main_release_name: the title of the main_release of the master
    :return: null
    """
    if cur_release.id in cur_album.version_ids:
        return
    else:
        cur_album.version_ids.add(cur_release.id)
    is_release_main_artist = False
    is_in_credits = False

    has_track_specific_info_in_credits = False
    for main_artist_e in cur_release.artists:
        if main_artist_e.id == artist_id:
            is_release_main_artist = True
            break
    for credit_artist_e in cur_release.credits:
        if credit_artist_e.id == artist_id:
            is_in_credits = True
            break
    if is_in_credits:        
        for extra_artist_entry in cur_release.data['extraartists']:
            if extra_artist_entry['name'] == artist_name and extra_artist_entry['tracks'] != "":
                new_str_id = extra_artist_entry['role'] + " -> tracks: " + extra_artist_entry['tracks']
                if not new_str_id in cur_album.track_specific_info:
                    cur_album.track_specific_info.add(new_str_id)
                    has_track_specific_info_in_credits = True
    update_all_cur_version = is_release_main_artist or is_in_credits

    if has_track_specific_info_in_credits or ( update_all_cur_version and not updated_all_in_main_release ):
        for track_entry in cur_release.tracklist:
            create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album)
    elif updated_all_in_main_release:
        return
    else:
        for track_entry in cur_release.tracklist:
            cur_str = track_entry.title + " in " + main_release_name
            if not cur_str in added_track_set:
                check_single_track_credit(cur_release, track_entry, artist_obj, is_release_main_artist, is_in_credits, main_release_name, cur_album)

# Main program starts here
for cur_rel in greg_rel:
    # Each release's type is other master or release
    if cur_rel.data["type"] == "master":
        main_release_tmp = cur_rel.main_release
        # We first check the main_release of the master
        tmp_album, should_update_all = get_info_from_release(main_release_tmp, main_release_tmp.title.strip())
        # Then we iterate through other versions, mainly to collect new Countries of Release, Catalog #, Year,
        tmp_versions = cur_rel.versions
        for version_entry in tmp_versions:
            update_from_other_release_version(version_entry, should_update_all, main_release_tmp.title.strip(), tmp_album)
    else:
        tmp_album, boolean_never_used = get_info_from_release(cur_rel, cur_rel.title.strip())
    tmp_album.tracklist.sort(key=lambda e: (e.title, e.release_year, e.release_id))
    print(tmp_album.tracklist)


[Funk It (Funkadelala) Blast! 1982, Stomp! Blast! 1982, The Real Thing Blast! 1982]
[All Of You So Excited! 0, All Of You So Excited! 1982, All Of You So Excited! 1982, All Of You So Excited! 1982, American Music So Excited! 0, American Music So Excited! 1982, American Music So Excited! 1982, American Music So Excited! 1982, Heart Beat So Excited! 0, Heart Beat So Excited! 1982, Heart Beat So Excited! 1982, Heart To Heart So Excited! 0, Heart To Heart So Excited! 1982, Heart To Heart So Excited! 1982, Heart To Heart So Excited! 1982, Heartbeat So Excited! 1982, I Feel For You So Excited! 0, I Feel For You So Excited! 1982, I Feel For You So Excited! 1982, I Feel For You So Excited! 1982, I'm So Excited So Excited! 0, I'm So Excited So Excited! 1982, I'm So Excited So Excited! 1982, I'm So Excited So Excited! 1982, If You Wanna Get Back Your Lady So Excited! 0, If You Wanna Get Back Your Lady So Excited! 1982, If You Wanna Get Back Your Lady So Excited! 1982, If You Wanna Get Back Your 

KeyboardInterrupt: 